#BUILD THE NEURAL NETWORK

Neural networks comprise (*meliputi*) of layers/modules that perform operations on data. The torch.nn namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

#Get Device for Training

We want to be able to train our model on a hardware accelerator like the GPU, if it is available. Let’s check to see if torch.cuda is available, else we continue to use the CPU

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


#Define the Class

We define our neural network by subclassing nn.Module, and initialize the neural network layers in \_\_init__. Every nn.Module subclass implements the operations on input data in the forward method.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of NeuralNetwork, and move it to the device, and print its structure.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class. We get the prediction probabilities by passing it through an instance of the nn.Softmax module.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5], device='cuda:0')


#Model Layers

Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


#nn.Flatten

We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous (*berdampingan*) array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#nn.Linear

The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#nn.ReLU

Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2811,  0.3467, -0.0308, -0.0173, -0.5731, -0.2356, -0.0445, -0.6779,
          0.2514, -0.0177,  0.1619,  0.0618, -0.2688, -0.1989,  0.2750,  0.2829,
         -0.2101, -0.2271, -0.0264, -0.4615],
        [-0.1725,  0.1511,  0.0703,  0.3816, -0.6457, -0.3524, -0.1272, -0.2332,
          0.2467, -0.1513,  0.2591,  0.3341, -0.2855, -0.2286,  0.5074, -0.1370,
         -0.3176, -0.2279,  0.1765, -0.2105],
        [-0.4628,  0.1198,  0.1838,  0.2343, -0.7568, -0.4922, -0.2142, -0.2998,
          0.4057, -0.3200,  0.0677,  0.0609, -0.5708, -0.6184,  0.0207,  0.0376,
         -0.4419, -0.3334, -0.0373, -0.1311]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.3467, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2514,
         0.0000, 0.1619, 0.0618, 0.0000, 0.0000, 0.2750, 0.2829, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.1511, 0.0703, 0.3816, 0.0000, 0.0000, 0.0000, 0.0000, 0.2467,
         0.0000, 0.2591, 0.3341, 0.0000, 0.0000, 0.50

#nn.Sequential

nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

#nn.Softmax

The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

#Model Parameters

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [12]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0077, -0.0094, -0.0175,  ...,  0.0145,  0.0229, -0.0213],
        [ 0.0326, -0.0041, -0.0209,  ...,  0.0281, -0.0128, -0.0128]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0043, -0.0213], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0150, -0.0229,  0.0415,  ..., -0.0399, -0.0229,  0.0315],
        [ 0.0079, -0.0328, -0.0003,  ...,  0.0397,  0.0192, -0.0387]],
       device='cuda:0', grad_fn=<